In [2]:
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
from prompts import CV_PROMPT, CV_COT_PROMPT

def process_judgment(judgment_str: str) -> str:
    # First try to find the exact \boxed{letter} pattern
    boxed_matches = re.findall(r'boxed{([A-C])}', judgment_str)
    if boxed_matches:
        return boxed_matches[-1]
    
    # Directly return the judgment if it is A, B, or C
    if judgment_str in ["A", "B", "C"]:
        return judgment_str
    else:
        final_judgment_str = judgment_str.split("Final Judgment:")[-1]
        matches = re.findall(r'\(([A-C])\)*', final_judgment_str)
        if matches:
            return matches[-1]
        matches = re.findall(r'([A-C])', final_judgment_str)
        if matches:
            return matches[-1]
        return ""

model_name = "opencompass/CompassVerifier-3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

/fs-computility/llmeval/liuhongwei/miniconda3/envs/oc_vllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.52s/it]


In [40]:
test_sample = [
    {
        "question": "Harold tosses a nickel four times.  What is the probability that he gets at least as many heads as tails?",
        "gold_answer": "\\frac{11}{16}",
        "llm_response": "Alright, I have this probability problem to solve:\n\n**Problem:** Harold tosses a nickel four times. What is the probability that he gets at least as many heads as tails?\n\nFirst, I need to understand what's being asked. Harold is tossing a nickel (which I assume is a fair coin) four times. I need to find the probability that the number of heads is at least as many as the number of tails.\n\n### Understanding the Problem\n\n- **Total number of tosses (n):** 4\n- **Coin:** Fair, so probability of heads (H) = probability of tails (T) = 0.5\n- **Desired outcome:** Number of heads ≥ number of tails\n\nSince there are four tosses, the number of heads can range from 0 to 4. The number of tails will then be 4 minus the number of heads.\n\nSo, the possible number of heads (k) and corresponding number of tails (4 - k) are:\n\n- k = 0: Tails = 4\n- k = 1: Tails = 3\n- k = 2: Tails = 2\n- k = 3: Tails = 1\n- k = 4: Tails = 0\n\nWe need the number of heads to be at least as many as the number of tails, i.e., k ≥ 4 - k.\n\nLet's solve the inequality:\n\nk ≥ 4 - k  \nk + k ≥ 4  \n2k ≥ 4  \nk ≥ 2\n\nSo, the number of heads must be 2, 3, or 4.\n\n### Calculating the Probabilities\n\nTo find the probability of getting at least as many heads as tails, I need to calculate the probabilities for k = 2, 3, and 4 and then sum them up.\n\nThe probability of getting exactly k heads in n tosses of a fair coin is given by the binomial probability formula:\n\nP(k) = C(n, k) * (0.5)^n\n\nWhere:\n- C(n, k) is the number of combinations of n items taken k at a time.\n- (0.5)^n is the probability of any specific sequence of n tosses.\n\nSince the coin is fair, (0.5)^n will be the same for all k, so we can factor that out when summing the probabilities.\n\nBut to simplify, I'll calculate each probability separately.\n\n#### 1. Probability of getting exactly 2 heads (k = 2)\n\nC(4, 2) = 6  \nP(2) = 6 * (0.5)^4 = 6 * 0.0625 = 0.375\n\n#### 2. Probability of getting exactly 3 heads (k = 3)\n\nC(4, 3) = 4  \nP(3) = 4 * (0.5)^4 = 4 * 0.0625 = 0.25\n\n#### 3. Probability of getting exactly 4 heads (k = 4)\n\nC(4, 4) = 1  \nP(4) = 1 * (0.5)^4 = 1 * 0.0625 = 0.0625\n\n### Summing the Probabilities\n\nNow, I'll sum the probabilities for k = 2, 3, and 4:\n\nP(at least as many heads as tails) = P(2) + P(3) + P(4)  \n= 0.375 + 0.25 + 0.0625  \n= 0.6875\n\n### Verifying the Calculation\n\nTo ensure I didn't make a mistake, I'll double-check the calculations:\n\n- **C(4, 2) = 6** ✔️\n- **C(4, 3) = 4** ✔️\n- **C(4, 4) = 1** ✔️\n- **(0.5)^4 = 0.0625** ✔️\n- **0.375 + 0.25 + 0.0625 = 0.6875** ✔️\n\nEverything seems to add up correctly.\n\n### Final Answer\n\nThe probability that Harold gets at least as many heads as tails when tossing a nickel four times is:\n\n\\boxed{0.6875}",
        "gold_judgment": "A"
    },
    {
        "question": "Factor completely over the set of polynomials with integer coefficients:\n\\[x^4 - 4x^3 + 14x^2 - 4x + 13.\\]",
        "gold_answer": "(x^2 + 1)(x^2 - 4x + 13)",
        "llm_response": "To factor the polynomial $x^4 - 4x^3 + 14x^2 - 4x + 13$ completely over the set of polynomials with integer coefficients, we will follow these steps:\n\n1. **Check for possible rational roots**: Use the Rational Root Theorem, which states that any rational root, expressed as a fraction $\\frac{p}{q}$, must have $p$ as a factor of the constant term (13) and $q$ as a factor of the leading coefficient (1). The possible rational roots are $\\pm1, \\pm13$.\n\n2. **Test the possible rational roots**: Substitute each possible rational root into the polynomial to check if it equals zero.\n\nLet's start by testing $x = 1$:\n$$\n1^4 - 4 \\cdot 1^3 + 14 \\cdot 1^2 - 4 \\cdot 1 + 13 = 1 - 4 + 14 - 4 + 13 = 20 \\neq 0\n$$\nSo, $x = 1$ is not a root.\n\nNext, test $x = -1$:\n$$\n(-1)^4 - 4 \\cdot (-1)^3 + 14 \\cdot (-1)^2 - 4 \\cdot (-1) + 13 = 1 + 4 + 14 + 4 + 13 = 36 \\neq 0\n$$\nSo, $x = -1$ is not a root.\n\nNext, test $x = 13$:\n$$\n13^4 - 4 \\cdot 13^3 + 14 \\cdot 13^2 - 4 \\cdot 13 + 13\n$$\nThis value is extremely large and clearly not zero.\n\nNext, test $x = -13$:\n$$\n(-13)^4 - 4 \\cdot (-13)^3 + 14 \\cdot (-13)^2 - 4 \\cdot (-13) + 13\n$$\nThis value is also extremely large and clearly not zero.\n\nSince none of the possible rational roots are actual roots, we proceed to other methods.\n\n3. **Grouping and factoring by grouping**: This method is not directly applicable here since the polynomial is a quartic and does not factor neatly into quadratic pairs.\n\n4. **Using substitution or known identities**: Sometimes, quartic polynomials can be transformed into quadratic polynomials using substitutions or identities. However, this polynomial does not appear to fit neatly into such categories.\n\n5. **Numerical or graphical methods**: For polynomials that do not factor easily, numerical methods or graphing can be used to approximate roots. However, since we are looking for integer coefficients, we need exact factors.\n\nGiven the complexity and the results from our initial checks, we conclude that the polynomial $x^4 - 4x^3 + 14x^2 - 4x + 13$ does not factor further over the set of polynomials with integer coefficients.\n\nThus, the final answer is:\n$$\n\\boxed{x^4 - 4x^3 + 14x^2 - 4x + 13}\n$$",
        "gold_judgment": "B",
    },
    {
        "question": "请你做一道数学选择题\n请你一步一步思考并将思考过程写在【解析】和<eoe>之间。你将从A，B，C，D中选出正确的答案，并写在【答案】和<eoa>之间。\n例如：【答案】: A <eoa>\n完整的题目回答的格式如下：\n【解析】 ... <eoe>\n【答案】 ... <eoa>\n请你严格按照上述格式作答。\n题目如下：8. （5 分) 设 $\\alpha \\in\\left(0, \\frac{\\pi}{2}\\right), \\beta \\in\\left(0, \\frac{\\pi}{2}\\right)$, 且 $\\tan \\alpha=\\frac{1+\\sin \\beta}{\\cos \\beta}$, 则（）\nA. $3 \\alpha-\\beta=\\frac{\\pi}{2}$\nB. $3 \\alpha+\\beta=\\frac{\\pi}{2}$\nC. $2 \\alpha-\\beta=\\frac{\\pi}{2}$\nD. $2 \\alpha+\\beta=\\frac{\\pi}{2}$\n\n",
        "gold_answer": "['C']",
        "llm_response": "【解析】\n首先，我们从给定的条件 $\\tan \\alpha = \\frac{1 + \\sin \\beta}{\\cos \\beta}$ 开始分析。\n\n1. 使用三角恒等式，我们可以将 $\\frac{1 + \\sin \\beta}{\\cos \\beta}$ 进行分解：\n   \\[\n   \\frac{1 + \\sin \\beta}{\\cos \\beta} = \\frac{1}{\\cos \\beta} + \\frac{\\sin \\beta}{\\cos \\beta} = \\sec \\beta + \\tan \\beta\n   \\]\n   因此，$\\tan \\alpha = \\sec \\beta + \\tan \\beta$。\n\n2. 我们知道 $\\sec \\beta = \\frac{1}{\\cos \\beta}$，所以可以将上式改写为：\n   \\[\n   \\tan \\alpha = \\frac{1}{\\cos \\beta} + \\frac{\\sin \\beta}{\\cos \\beta} = \\frac{1 + \\sin \\beta}{\\cos \\beta}\n   \\]\n   这与原条件一致。\n\n3. 接下来，我们需要验证选项中的关系。我们从选项 D 开始验证：$2 \\alpha + \\beta = \\frac{\\pi}{2}$。\n\n4. 如果 $2 \\alpha + \\beta = \\frac{\\pi}{2}$，则 $\\beta = \\frac{\\pi}{2} - 2 \\alpha$。\n\n5. 将 $\\beta = \\frac{\\pi}{2} - 2 \\alpha$ 代入原条件 $\\tan \\alpha = \\frac{1 + \\sin \\beta}{\\cos \\beta}$ 中：\n   \\[\n   \\sin \\beta = \\sin \\left( \\frac{\\pi}{2} - 2 \\alpha \\right) = \\cos 2 \\alpha\n   \\]\n   \\[\n   \\cos \\beta = \\cos \\left( \\frac{\\pi}{2} - 2 \\alpha \\right) = \\sin 2 \\alpha\n   \\]\n\n6. 代入后得到：\n   \\[\n   \\tan \\alpha = \\frac{1 + \\cos 2 \\alpha}{\\sin 2 \\alpha}\n   \\]\n\n7. 使用双角公式 $\\cos 2 \\alpha = 2 \\cos^2 \\alpha - 1$ 和 $\\sin 2 \\alpha = 2 \\sin \\alpha \\cos \\alpha$，我们得到：\n   \\[\n   \\tan \\alpha = \\frac{1 + 2 \\cos^2 \\alpha - 1}{2 \\sin \\alpha \\cos \\alpha} = \\frac{2 \\cos^2 \\alpha}{2 \\sin \\alpha \\cos \\alpha} = \\frac{\\cos \\alpha}{\\sin \\alpha} = \\cot \\alpha\n   \\]\n\n8. 由于 $\\tan \\alpha = \\cot \\alpha$，这意味着 $\\alpha = \\frac{\\pi}{4}$。\n\n9. 代入 $\\alpha = \\frac{\\pi}{4}$ 回到 $\\beta = \\frac{\\pi}{2} - 2 \\alpha$，我们得到：\n   \\[\n   \\beta = \\frac{\\pi}{2} - 2 \\cdot \\frac{\\pi}{4} = \\frac{\\pi}{2} - \\frac{\\pi}{2} = 0\n   \\]\n\n10. 但是 $\\beta$ 必须在 $(0, \\frac{\\pi}{2})$ 之间，因此 $\\alpha = \\frac{\\pi}{4}$ 不成立。\n\n11. 重新检查选项，发现选项 C 也可能是正确的。验证选项 C：$2 \\alpha - \\beta = \\frac{\\pi}{2}$。\n\n12. 如果 $2 \\alpha - \\beta = \\frac{\\pi}{2}$，则 $\\beta = 2 \\alpha - \\frac{\\pi}{2}$。\n\n13. 将 $\\beta = 2 \\alpha - \\frac{\\pi}{2}$ 代入原条件 $\\tan \\alpha = \\frac{1 + \\sin \\beta}{\\cos \\beta}$ 中：\n    \\[\n    \\sin \\beta = \\sin \\left( 2 \\alpha - \\frac{\\pi}{2} \\right) = -\\cos 2 \\alpha\n    \\]\n    \\[\n    \\cos \\beta = \\cos \\left( 2 \\alpha - \\frac{\\pi}{2} \\right) = \\sin 2 \\alpha\n    \\]\n\n14. 代入后得到：\n    \\[\n    \\tan \\alpha = \\frac{1 - \\cos 2 \\alpha}{\\sin 2 \\alpha}\n    \\]\n\n15. 使用双角公式 $\\cos 2 \\alpha =",
        "gold_judgment": "C"
    }
]

## For straight forward verification

In [45]:
for sample in test_sample:
    model_input = CV_PROMPT.format(question=sample["question"], gold_answer=sample["gold_answer"], llm_response=sample["llm_response"])
    messages=[{"role": "user", "content": model_input}]
    input_text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False
    )
    model_inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    output=model.generate(
        **model_inputs,
        do_sample=False,
        max_new_tokens=2048)
    judgement=tokenizer.decode(output[0][model_inputs.input_ids.shape[1]:],skip_special_tokens=True)

    print(judgement)
    print("*"*100)

A
****************************************************************************************************
B
****************************************************************************************************
C
****************************************************************************************************


## For COT verification

In [44]:
for sample in test_sample:
    model_input = CV_COT_PROMPT.format(question=sample["question"], gold_answer=sample["gold_answer"], llm_response=sample["llm_response"])
    messages=[{"role": "user", "content": model_input}]
    input_text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False
    )
    model_inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    output=model.generate(
        **model_inputs,
        do_sample=False,
        max_new_tokens=2048)
    judgement=tokenizer.decode(output[0][model_inputs.input_ids.shape[1]:],skip_special_tokens=True)

    print("ori_judgment: ", judgement)
    print("processed_judgment: ", process_judgment(judgement))
    print("*"*100)

/fs-computility/llmeval/liuhongwei/miniconda3/envs/oc_vllm/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/fs-computility/llmeval/liuhongwei/miniconda3/envs/oc_vllm/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/fs-computility/llmeval/liuhongwei/miniconda3/envs/oc_vllm/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sampl

ori_judgment:  Analysis step by step:

1. **Validity Check**:
   - The candidate's answer is complete, not repetitive, and not a refusal. It provides a detailed explanation and arrives at a final answer.

2. **Question Requirements**:
   - The question asks for the probability that Harold gets at least as many heads as tails when tossing a nickel four times. The standard answer is \(\frac{11}{16}\), which is approximately 0.6875.

3. **Comparison of Answers**:
   - The candidate's final answer is \(\boxed{0.6875}\).
   - The standard answer is \(\frac{11}{16}\), which is numerically equivalent to 0.6875.
   - The candidate's answer matches the standard answer in both value and meaning, despite the difference in presentation (decimal vs. fraction).

4. **Numerical Precision**:
   - The candidate's answer is precise and matches the standard answer when converted to decimal form.

Final Judgment: \boxed{A} - CORRECT
processed_judgment:  A
**************************************************